FAISS

In [ ]:
!pip install faiss-cpu google-generativeai


In [ ]:
!pip install faiss-cpu google-generativeai



In [ ]:
import faiss
import numpy as np
import google.generativeai as genai

# Configure the API key directly
genai.configure(api_key="Your API key here")
embedding_model = genai.get_model("models/embedding-001")
embedding_dim = 768
faiss_index = faiss.IndexFlatL2(embedding_dim)
vector_store_texts = []

In [ ]:
def add_to_vector_store(text):
    embedding = embedding_model.embed(content=text)["embedding"]
    faiss_index.add(np.array([embedding], dtype='float32'))
    vector_store_texts.append(text)


In [ ]:
def search_vector_store(query, top_k=3):
    embedding = embedding_model.embed(content=query)["embedding"]
    D, I = faiss_index.search(np.array([embedding], dtype='float32'), top_k)
    results = [vector_store_texts[i] for i in I[0] if i < len(vector_store_texts)]
    return results


In [ ]:
def get_gemini_reply(user_input):
    relevant_contexts = search_vector_store(user_input)
    context_text = "\n".join(relevant_contexts)

    prompt_with_context = f"Context:\n{context_text}\n\nUser: {user_input}"

    conversation_history.append(types.Content(role="user", parts=[types.Part(text=prompt_with_context)]))

    response = client.models.generate_content(
        model="gemini-2.0-flash",
        contents=conversation_history,
        config=config_with_search,
    )
    reply_text = response.candidates[0].content.parts[0].text

    # Store input and response in FAISS
    add_to_vector_store(user_input)
    add_to_vector_store(reply_text)

    conversation_history.append(types.Content(role="model", parts=[types.Part(text=reply_text)]))
    return reply_text


In [ ]:
import google.generativeai as genai
from IPython.display import display, Markdown
from google.generativeai.types import GenerationConfig

# Configure API
GOOGLE_API_KEY = "Your api key here"
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel("gemini-2.0-flash")

# Dummy vector store functions
def search_vector_store(query):
    return ["Climate change refers to long-term shifts in temperatures and weather patterns."]

def add_to_vector_store(text):
    pass
config_with_search = GenerationConfig(
    temperature=0.7,
    top_p=1,
    top_k=1,
    max_output_tokens=1024
)

# Simulated user input
user_input = "Explain the causes of climate change."
relevant_contexts = search_vector_store(user_input)
context_text = "\n".join(relevant_contexts)
prompt_with_context = f"Context:\n{context_text}\n\nUser: {user_input}"

# Build history with plain dicts
conversation_history = []
conversation_history.append({
    "role": "user",
    "parts": [prompt_with_context]
})

# Call Gemini
try:
    response = model.generate_content(
        contents=conversation_history,
        generation_config=config_with_search
    )
    reply_text = response.text
    display(Markdown(reply_text))

    # Store in vector DB
    add_to_vector_store(user_input)
    add_to_vector_store(reply_text)

    # Add model response to history
    conversation_history.append({
        "role": "model",
        "parts": [reply_text]
    })

except Exception as e:
    print("Error:", e)


In [ ]:
def chat_with_gemini(user_input):
    # 1. Search context
    relevant_contexts = search_vector_store(user_input)
    context_text = "\n".join(relevant_contexts)
    prompt_with_context = f"Context:\n{context_text}\n\nUser: {user_input}"
    conversation_history.append(
        Content(role="user", parts=[Part(text=prompt_with_context)])
    )
    response = model.generate_content(
        contents=conversation_history,
        generation_config=generation_config
    )
    reply_text = response.text
    display(Markdown(f"**Gemini:** {reply_text}"))

    add_to_vector_store(user_input)
    add_to_vector_store(reply_text)

    # 6. Append model response to history
    conversation_history.append(
        Content(role="model", parts=[Part(text=reply_text)])
    )


Vector db and MCP Chatbot

In [ ]:
# ✅ Install requirements
!pip install -q google-generativeai faiss-cpu

# === Imports ===
import google.generativeai as genai
import faiss
import numpy as np
from IPython.display import display, Markdown

# === API Setup ===
GOOGLE_API_KEY = "Your api key here"  # Replace with your actual API key
genai.configure(api_key=GOOGLE_API_KEY)

# === Vector Store Setup ===
embedding_dimension = 768
index = faiss.IndexFlatIP(embedding_dimension)
texts = []

def get_embedding(text):
    res = genai.embed_content(
        model="models/embedding-001",
        content=text,
        task_type="retrieval_document"
    )
    emb = np.array(res['embedding'], dtype="float32")
    return emb / np.linalg.norm(emb)

def add_to_vector_store(text):
    emb = get_embedding(text)
    index.add(np.expand_dims(emb, axis=0))
    texts.append(text)

def search_vector_store(query, top_k=3):
    if index.ntotal == 0:
        return []
    emb = get_embedding(query)
    D, I = index.search(np.expand_dims(emb, axis=0), top_k)
    return [texts[i] for i in I[0] if i < len(texts)]

# === Chat Setup ===
model = genai.GenerativeModel("gemini-2.0-flash")

generation_config = {
    "temperature": 0.7,
    "max_output_tokens": 500,
    "top_p": 0.95,
    "top_k": 40
}

conversation_history = []

def chat_with_gemini(user_input):
    retrieved = search_vector_store(user_input)
    context_text = "\n".join(retrieved)

    prompt = f"Context:\n{context_text}\n\nUser: {user_input}"

    conversation_history.append({
        "role": "user",
        "parts": [{"text": prompt}]
    })

    response = model.generate_content(
        contents=conversation_history,
        generation_config=generation_config
    )

    reply_text = response.text

    display(Markdown(f"**Gemini:** {reply_text}"))

    add_to_vector_store(user_input)
    add_to_vector_store(reply_text)

    conversation_history.append({
        "role": "model",
        "parts": [{"text": reply_text}]
    })

# === Usage ===
print("Gemini chatbot ready. Type your message (or 'exit' to quit):")

while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        print("Goodbye!")
        break
    chat_with_gemini(user_input)


PInecone

In [ ]:
!pip install google-generativeai pinecone

In [ ]:
!pip install --upgrade pinecone

In [ ]:
import google.generativeai as genai
from pinecone import Pinecone, ServerlessSpec
import numpy as np

# Configure Gemini
genai.configure(api_key="Your API key here")
model = genai.GenerativeModel("gemini-2.0-flash")

# Refer to Pinecone documentation for the correct initialization for ServerlessSpec
pc = Pinecone(api_key="Your pinecone api key")
index = pc.Index("gemini-chat")
namespace = "chat"

def embed(text):
    return model.embed_content(
        content=text,
        task_type="retrieval_document",
        title="user input"
    )['embedding']

# Add to vector DB
def add_to_pinecone(text, id_):
    vector = embed(text)
    index.upsert(vectors=[{"id": id_, "values": vector}], namespace=namespace)

# Search vector DB
def search_pinecone(query, top_k=5):
    vector = embed(query)
    res = index.query(vector=vector, top_k=top_k, namespace=namespace, include_metadata=True)
    return [match['metadata'].get('text', '') for match in res['matches']]

# Chat logic
conversation = []

def chat(user_input):
    results = search_pinecone(user_input)
    context = "\n".join(results)
    prompt = f"Context:\n{context}\n\nUser: {user_input}"
    conversation.append({"role": "user", "parts": [prompt]})

    response = model.generate_content(conversation)
    reply = response.text
    print("Gemini:", reply)

    conversation.append({"role": "model", "parts": [reply]})
    add_to_pinecone(user_input, f"user-{len(conversation)}")
    add_to_pinecone(reply, f"model-{len(conversation)}")

Qdrant

In [ ]:
!pip install qdrant-client google-generativeai

In [ ]:
!pip install qdrant-client google-generativeai


In [ ]:
import uuid

# Generate a random UUID (version 4)
new_uuid = str(uuid.uuid4())

print(new_uuid)


In [ ]:
# Install dependencies if needed (uncomment if running in a fresh Colab session)
!pip install google-generativeai qdrant-client

embedding_model = genai.GenerativeModel("embed-text-bison@001")


In [ ]:
def embed(text):
    response = genai.embed_text(text)
    # response is a dict with embeddings under "embedding"
    return response["embedding"]


In [ ]:
import google.generativeai as genai
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct
import uuid

# Configure API key
genai.configure(api_key="your api key here")

# Chat model
chat_model = genai.GenerativeModel("gemini-2.0-flash")

# Qdrant setup
qdrant = QdrantClient(":memory:")

if not qdrant.collection_exists("chat"):
    qdrant.create_collection(
        collection_name="chat",
        vectors_config=VectorParams(size=768, distance=Distance.COSINE)  # Use 768 or 512 depending on embedding size; test to confirm
    )

# Embedding helper - use top-level embed_content function
def embed(text):
    # Use genai.embed_content instead of genai.embed_text
    response = genai.embed_content(
        model="models/embedding-001",  # Specify the embedding model
        content=text,
        task_type="retrieval_document" # Specify the task type
    )
    return response["embedding"]

def add_to_qdrant(text):
    vector = embed(text)
    unique_id = str(uuid.uuid4())
    qdrant.upsert(
        collection_name="chat",
        points=[PointStruct(id=unique_id, vector=vector, payload={"text": text})]
    )

def search_qdrant(query, top_k=5):
    vector = embed(query)
    hits = qdrant.search(collection_name="chat", query_vector=vector, limit=top_k)
    return [hit.payload["text"] for hit in hits]

conversation = []

def chat(user_input):
    results = search_qdrant(user_input)
    context = "\n".join(results)
    prompt = f"Context:\n{context}\n\nUser: {user_input}"
    conversation.append({"role": "user", "parts": [prompt]})

    response = chat_model.generate_content(conversation)
    reply = response.text
    print("Gemini:", reply)

    conversation.append({"role": "model", "parts": [reply]})
    add_to_qdrant(user_input)
    add_to_qdrant(reply)

# Run loop
print("🤖 Gemini Chatbot (type 'exit' to quit)")
while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("Goodbye!")
        break
    chat(user_input)

Weaviate

In [ ]:
!pip install weaviate-client --upgrade
!pip install google-generativeai


In [ ]:
!pip install --upgrade weaviate-client


In [ ]:
# STEP 1: Install required libraries
!pip install -q google-generativeai weaviate-client

# STEP 2: Import modules
import google.generativeai as genai
import weaviate
from weaviate.classes.config import Configure, DataType
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import os

# STEP 3: Gemini API Key setup
genai.configure(api_key="your api key here")
model = genai.GenerativeModel("gemini-2.0-flash")

# STEP 4: Connect to Weaviate Cloud (make sure Weaviate project is set up at https://console.weaviate.io)
WEAVIATE_URL = "place your weaviate url"  # e.g., "abc1234.weaviate.network"
WEAVIATE_API_KEY = "place weaviate api key"

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=weaviate.auth.AuthApiKey(WEAVIATE_API_KEY),
    headers={"X-OpenAI-Api-Key": WEAVIATE_API_KEY}
)

# STEP 5: Create collection for chat memory if not exists
if "ChatMemory" not in client.collections.list_all():
    client.collections.create(
        name="ChatMemory",
        properties=[{"name": "text", "dataType": DataType.TEXT}],
        vectorizer_config=Configure.Vectorizer.none()  # No built-in vectorizer
    )

collection = client.collections.get("ChatMemory")
# STEP 6: Embedding function using Gemini
def embed(text):
    # Call genai.embed_content directly, not on the model object
    res = genai.embed_content(
        model="models/embedding-001", # Specify the embedding model
        content=text,
        task_type="RETRIEVAL_DOCUMENT"
    )
    return np.array(res["embedding"], dtype=np.float32)

# STEP 7: Add text to Weaviate memory
def add_to_memory(text):
    vector = embed(text)
    collection.data.insert(properties={"text": text}, vector=vector)

# STEP 8: Search Weaviate memory for relevant context
def search_memory(query, top_k=3):
    vector = embed(query)
    # Corrected call to near_vector: pass vector as positional argument
    results = collection.query.near_vector(vector, limit=top_k)
    return [o.properties["text"] for o in results.objects]

# STEP 9: Conversational chatbot function
def chat(user_input, history=[]):
    context = search_memory(user_input)
    context_str = "\n".join(context)

    prompt = f"""You are a helpful assistant. Use the following context to answer the user's question.

Context:
{context_str}

User: {user_input}
"""

    response = model.generate_content(prompt)
    reply = response.text

    # Store user input and model reply
    add_to_memory(user_input)
    add_to_memory(reply)

    history.append((user_input, reply))
    return reply

# STEP 10: Run chatbot loop
print("🤖 Gemini Chatbot (type 'exit' to quit)\n")
chat_history = []

while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("Goodbye!")
        break
    response = chat(user_input, chat_history)
    print(f"Assistant: {response}\n")